# Emotion/valence/arousal reconstructions
a.k.a. multivariate reverse correlation.

In [ ]:
import sys
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pymc3 as pm
import theano.tensor as tt

sys.path.append('../src')
from utils import plot_face, get_parameters

Define some parameters to make the code look nice.

In [ ]:
N = 848  # number of unique stimuli
n_v = 31049  # number of vertices

# Load vertices, both static and dynamic
v = np.load('../data/vertices.npz')['v']
vs = v[:, 0, :, :]
vd = v[:, 1, :, :]

mean_face = vs.mean(axis=0)  # for plotting
tris = np.load('../data/tris.npy') - 1  # triangles

## Dynamic face information reconstruction
We fit a logistic regression model that uses PCA components of dynamic face information (vertex-based deformation between peak frame and start frame), $X$, to predict a categorical emotion label, $y$:

\begin{align}
p(y | X) = \mathrm{softmax}(X\beta + \alpha)
\end{align}

This was done using scikit-learn. We load in the coefficients ("beta_hat_vd") and the offset ("alpha_hat_vd") below.

In [ ]:
coef = pd.read_csv('../results/validation/target-emotion_fs-vertexPCA_type-dynamic_coefs.tsv', sep='\t', index_col=0)
SUB = 'average'
alpha_hat, beta_hat, Z = get_parameters(SUB, coef)

Now, if we want to know the face movement in vertex space that, according to our previously estimated logistic regression model, predicts a particular emotion label, i.e., $p(X|Y)$, we need to invert the model. We can do this using Bayesian modelling with *pymc3*:

In [ ]:
from model import run_inverted_logreg
run_inverted_logreg??

In [ ]:
# Load in the features in PCA space to determine the range of the uniform prior
vd_pca = pd.read_csv('../data/features/vertexPCA_type-dynamic.tsv', sep='\t', index_col=0).to_numpy()
trace, X_hdp = run_inverted_logreg(vd_pca, beta_hat, alpha_hat, return_hdp=True)

Now, the posterior distribution of our $X$ variable represents the probability distribution of our PCA components, given a particular emotion ("anger", "happiness", etc.). The function also spits out `X_hdp`, a selection of values from the highest density region of our posterior.

In [ ]:
X_hdp.shape  # most likely values for 50 components for each of 6 emotions
np.savez('../results/validation/target-emotion_type-dynamic_posterior.npz', trace=trace, hdp=X_hdp)

Now onto the cool stuff. We can visualize the face that is most likely interpreted as a particular emotion. To do so, we need to invert the PCA operation to go from PCA space ($X$) to stimulus space ($S$). 

In [ ]:
with np.load('../results/pca/pca_type-dynamic_weights.npz') as data:
    # "vd" stands for "vertex difference"
    mu_vd, w_vd = data['mu'], data['W']

In [ ]:
S_emo_vd = ((X_hdp * vd_pca.std(axis=0)) @ w_vd + mu_vd).reshape((6, n_v, 3))
overlay_emo_vd = S_emo_vd / vd.std(axis=0)
overlay_emo_vd[np.isnan(overlay_emo_vd)] = 0
np.savez('../results/validation/target-emotion_type-dynamic_recon.npz', S=S_emo_vd, overlay=overlay_emo_vd)

In [ ]:
fig = plot_face(mean_face + S_emo_vd[0, :, :], tris, overlay=overlay_emo_vd[0, :, :], cmax=4, cmin=-4, threshold=0.1)
fig.show()

### Static

In [ ]:
# First PC
with np.load('../results/pca/pca_type-static_weights.npz') as data:
    mu_vs, w_vs = data['mu'], data['W']

vs_pca = pd.read_csv('../data/features/vertexPCA_type-static.tsv', sep='\t', index_col=0).to_numpy()

In [ ]:
coef = pd.read_csv('../results/validation/target-emotion_fs-vertexPCA_type-static_coefs.tsv', sep='\t', index_col=0)
alpha_hat, beta_hat, Z = get_parameters(SUB, coef)
# Note to self: leave out comp 50, because there's no variance in that (std = 0 -> NaN)
beta_hat = beta_hat[:, :49]

In [ ]:
trace, X_hdp = run_inverted_logreg(vs_pca[:, :49], beta_hat, alpha_hat, return_hdp=True)
np.savez('../results/validation/target-emotion_type-static_posterior.npz', trace=trace, X_hdp=X_hdp)

In [ ]:
S_emo_vs = ((X_hdp * vs_pca.std(axis=0)[:49]) @ w_vs[:49, :]).reshape((6, n_v, 3))
overlay_emo_vs = S_emo_vs / vs.std(axis=0)
overlay_emo_vs[np.isnan(overlay_emo_vs)] = 0
np.savez('../results/validation/target-emotion_type-static_recon.npz', S=S_emo_vs, overlay=overlay_emo_vs)

In [ ]:
#fig = plot_face(mu_vs.reshape((n_v, 3)) + S_emo_vs, tris, overlay=overlay_emo_vs, cmax=4, cmin=-4, threshold=0.1)
#fig.show()
#fig.write_image('../figures/target-emotion_type-static_recon.png', scale=2)

## Valence (dynamic)

In [ ]:
SUB = 'average'
coef = pd.read_csv('../results/validation/target-valence_fs-vertexPCA_type-dynamic_coefs.tsv', sep='\t', index_col=0)
alpha_hat, beta_hat, sigma_hat, Z = get_parameters(SUB, coef)

In [ ]:
from model import run_inverted_linreg
trace, X_hdp = run_inverted_linreg(vd_pca, beta_hat, alpha_hat, sigma_hat, return_hdp=True)
np.savez('../results/validation/target-valence_type-dynamic_posterior.npz', trace=trace, X_hdp=X_hdp)

In [ ]:
S_valence_vd = ((X_hdp * vd_pca.std(axis=0)) @ w_vd + mu_vd).reshape((7, n_v, 3))
overlay_valence_vd = S_valence_vd / vd.std(axis=0)
np.savez('../results/validation/target-valence_type-dynamic_recon.npz', S=S_valence_vd, overlay=overlay_valence_vd)

In [ ]:
fig = plot_face(mean_face + S_valence_vd[5, :, :], tris, overlay=overlay_valence_vd[5, :, :], cmax=4, cmin=-4, threshold=0.1)
fig.show()
#fig.write_image('../figures/target-valence_type-dynamic_recon.png', scale=2)

## Valence (static)

In [ ]:
coef = pd.read_csv('../results/validation/target-valence_fs-vertexPCA_type-static_coefs.tsv', sep='\t', index_col=0)
alpha_hat, beta_hat, sigma_hat, Z = get_parameters(SUB, coef)
beta_hat = beta_hat[:49]
trace, X_hdp = run_inverted_linreg(vs_pca[:, :49], beta_hat, alpha_hat, sigma_hat, return_hdp=True)

In [ ]:
np.savez('../results/validation/target-valence_type-static_posterior.npz', trace=trace, X_hdp=X_hdp)

In [ ]:
S_valence_vs = ((X_hdp * vs_pca.std(axis=0)[:49]) @ w_vs[:49, :]).reshape((7, n_v, 3))
overlay_valence_vs = S_valence_vs / vs.std(axis=0)
np.savez('../results/validation/target-valence_type-static_recon.npz', S=S_valence_vs, overlay=overlay_valence_vs)

In [ ]:
fig = plot_face(mu_vs.reshape((n_v, 3)) + S_valence_vs[0, :, :], tris, overlay=overlay_valence_vs[0, :, :], cmax=4, cmin=-4, threshold=0.1)
fig.show()
#fig.write_image('../figures/target-valence_type-static_recon.png', scale=2)

## Arousal (dynamic)

In [ ]:
coef = pd.read_csv('../results/validation/target-arousal_fs-vertexPCA_type-dynamic_coefs.tsv', sep='\t', index_col=0)
alpha_hat, beta_hat, sigma_hat, Z = get_parameters(SUB, coef)
trace, X_hdp = run_inverted_linreg(vd_pca, beta_hat, alpha_hat, sigma_hat, return_hdp=True)

In [ ]:
np.savez('../results/validation/target-arousal_type-dynamic_posterior.npz', trace=trace, X_hdp=X_hdp)

S_arousal_vd = ((X_hdp * vd_pca.std(axis=0)) @ w_vd + mu_vd).reshape((7, n_v, 3))
overlay_arousal_vd = S_arousal_vd / vd.std(axis=0)
np.savez('../results/validation/target-arousal_type-dynamic_recon.npz', S=S_arousal_vd, overlay=overlay_arousal_vd)

In [ ]:
fig = plot_face(mean_face + S_arousal_vd[4, :, :], tris, overlay=overlay_arousal_vd[4, :, :], cmax=4, cmin=-4, threshold=0.1)
fig.show()
#fig.write_image('../figures/target-arousal_type-dynamic_recon.png', scale=2)

## Arousal (static)

In [ ]:
coef = pd.read_csv('../results/validation/target-arousal_fs-vertexPCA_type-static_coefs.tsv', sep='\t', index_col=0)
alpha_hat, beta_hat, sigma_hat, Z = get_parameters(SUB, coef)
beta_hat = beta_hat[:49]
trace, X_hdp = run_inverted_linreg(vs_pca[:, :49], beta_hat, alpha_hat, sigma_hat, return_hdp=True)

In [ ]:
np.savez('../results/validation/target-arousal_type-static_posterior.npz', trace=trace, X_hdp=X_hdp)

S_arousal_vs = ((X_hdp * vs_pca.std(axis=0)[:49]) @ w_vs[:49, :]).reshape((7, n_v, 3))
overlay_arousal_vs = S_arousal_vs / vs.std(axis=0)
np.savez('../results/validation/target-arousal_type-static_recon.npz', S=S_arousal_vs, overlay=overlay_arousal_vs)

In [ ]:
fig = plot_face(mu_vs.reshape((n_v, 3)) + S_arousal_vs[0, :, :], tris, overlay=overlay_arousal_vs[0, :, :], cmax=4, cmin=-4, threshold=0.1)
fig.show()